<a href="https://colab.research.google.com/github/zh1212121/NLP-Assn-2/blob/main/Assignment_2_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
!git clone https://github.com/zh1212121/NLP-Assn-2.git

fatal: destination path 'NLP-Assn-2' already exists and is not an empty directory.


In [24]:
#!pip install Torch==1.7.0
#!pip install tensorflow==2.3.0
#!pip install transformers==4.5.0
!pip install scikit-learn==0.21
#!pip install keras==2.3.1
!pip uninstall numpy -y
!pip install numpy==1.19.0
!pip install nltk

Uninstalling numpy-1.19.0:
  Successfully uninstalled numpy-1.19.0
  Using cached https://files.pythonhosted.org/packages/a8/31/e2c3eda7afe7dab08e1f24767b8e38ff2f30dc82bd74aa3a5324c550366a/numpy-1.19.0-cp37-cp37m-manylinux2010_x86_64.whl
ERROR: tensorflow 2.5.0 has requirement numpy~=1.19.2, but you'll have numpy 1.19.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


In [51]:
import numpy as np
import pandas as pd
import torch
#import tensorflow as tf
import re
import string
import nltk
#import transformers
import sklearn as sk
import matplotlib.pyplot as plt
import time
import pickle

from sklearn import model_selection, metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.metrics import classification_report

from google.colab import files

In [26]:
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('punkt')
from nltk.tokenize import word_tokenize
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [27]:
seed = 1234

In [28]:
print(np.__version__)
print(pd.__version__)
#print(tf.__version__)
#print(torch.__version__)
#print(transformers.__version__)
print(sk.__version__)

1.19.0
1.1.5
0.21.0


##import dataset

In [29]:
#import given datasets
path_exam = "https://raw.githubusercontent.com/zh1212121/NLP-Assn-2/main/Test-format.csv"
exam_raw = pd.read_csv(path_exam, header = None)

In [30]:
exam_raw.head()

,0
0,review 1 x y z
1,review 2 x y z
2,review 3 x y z
3,review 4 x y z
4,review 5 x y z


## data preprocessing

lowercase<br />
remove $<br />$<br />
remove url (http://)<br />
remove punctuation<br />
remove non-char words/special char<br />

remove stopwords (pending testing)<br />
lemmatization (pending testing)<br />
remove single char
reduce multiple whitespaces
normalize?

tokenize


In [31]:
#change text to lower case
def convert_to_lower(text):
  return text.lower()

In [32]:
#remove <br />
def remove_newline(text):
  return re.sub('<br />', '', text)

In [33]:
#remove http:// links
def remove_link(text):
  return re.sub(r'http[s]?://\S+\b', '', text)

In [34]:
#decontraction
def decontract(text):
  text = re.sub(r"won\'t", " will not", text)
  text = re.sub(r"won\'t've", " will not have", text)
  text = re.sub(r"can\'t", " can not", text)
  text = re.sub(r"don\'t", " do not", text)
    
  text = re.sub(r"can\'t've", " can not have", text)
  text = re.sub(r"ma\'am", " madam", text)
  text = re.sub(r"let\'s", " let us", text)
  text = re.sub(r"ain\'t", " am not", text)
  text = re.sub(r"shan\'t", " shall not", text)
  text = re.sub(r"sha\n't", " shall not", text)
  text = re.sub(r"o\'clock", " of the clock", text)
  text = re.sub(r"y\'all", " you all", text)

  text = re.sub(r"n\'t", " not", text)
  text = re.sub(r"n\'t've", " not have", text)
  text = re.sub(r"\'re", " are", text)
  text = re.sub(r"\'s", " is", text)
  text = re.sub(r"\'d", " would", text)
  text = re.sub(r"\'d've", " would have", text)
  text = re.sub(r"\'ll", " will", text)
  text = re.sub(r"\'ll've", " will have", text)
  text = re.sub(r"\'t", " not", text)
  text = re.sub(r"\'ve", " have", text)
  text = re.sub(r"\'m", " am", text)
  text = re.sub(r"\'re", " are", text)
  return text

#reference:
#https://www.kaggle.com/faressayah/sentiment-model-with-tensorflow-transformers

In [35]:
sample = "I didn't expect to like this film as much as I did. I got it simply because I saw it on the list of Top 25 Most Controversial Films of All Time. It didn't look particularly great. I was pleasantly surprised to find that it was one of the most cleverly composed films of recent memory.<br /><br />It's about a twenty-year-old woman wants to know everything. She stores every bit of information she collects in an enormous archive. She experiments with experience in sex, political activism, and human relationships. Meanwhile, film's crew is shown making the film and we view their reactions to the story and each other. Nudity, explicit sex, and controversial politics kept this film from being shown in the US while its seizure by Customs was appealed. The film is a narrative yet it's a documentary that shows us the behind-the- scenes world of the filmmakers during the narrative, the fourth wall being broken. This film is the most direct possible way of making a movie I have ever seen. The movie predominantly works as a time capsule of 1960s psychedelic goings-on, freedom-fighting and sexual liberation. I like to think of it as much more than that.<br /><br />I didn't think I would want to waste my time with the blue version of this movie, but I actually really do. This film is a buried treasure. Give it a try."

decontract(sample)

'I did not expect to like this film as much as I did. I got it simply because I saw it on the list of Top 25 Most Controversial Films of All Time. It did not look particularly great. I was pleasantly surprised to find that it was one of the most cleverly composed films of recent memory.<br /><br />It is about a twenty-year-old woman wants to know everything. She stores every bit of information she collects in an enormous archive. She experiments with experience in sex, political activism, and human relationships. Meanwhile, film is crew is shown making the film and we view their reactions to the story and each other. Nudity, explicit sex, and controversial politics kept this film from being shown in the US while its seizure by Customs was appealed. The film is a narrative yet it is a documentary that shows us the behind-the- scenes world of the filmmakers during the narrative, the fourth wall being broken. This film is the most direct possible way of making a movie I have ever seen. Th

In [36]:
#remove punctuation
print(string.punctuation)

punctuation_defined = '\"#%\'()*+-//<=>@[\]^_`{|}'
print(punctuation_defined)

def remove_punc(text):
  return text.translate(str.maketrans('', '', punctuation_defined))
  #  return text.translate(str.maketrans('', '', string.punctuation))

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
"#%'()*+-//<=>@[\]^_`{|}


In [37]:
#remove special characters & numbers
def remove_special(text, remove_punc = True, remove_digits=True):
  if remove_punc:
    if remove_digits:
      pattern=r'[^a-zA-z\s]'
      text=re.sub(pattern,'',text)
    else:
      pattern=r'[^a-zA-z0-9\s]'
      text=re.sub(pattern,'',text)
  else:
    if remove_digits:
      pattern=r'[^a-zA-z\"#%\'()*+-//<=>@[\]^_`{|}\s]'
      text=re.sub(pattern,'',text)
    else:
      pattern=r'[^a-zA-z0-9\"#%\'()*+-//<=>@[\]^_`{|}\s]'
      text=re.sub(pattern,'',text)
  return text

In [38]:
sample = "I didn't expect to like this film as much as I did. I got it simply because I saw it on the list of Top 25 Most Controversial Films of All Time. It didn't look particularly great. I was pleasantly surprised to find that it was one of the most cleverly composed films of recent memory.<br /><br />It's about a twenty-year-old woman wants to know everything. She stores every bit of information she collects in an enormous archive. She experiments with experience in sex, political activism, and human relationships. Meanwhile, film's crew is shown making the film and we view their reactions to the story and each other. Nudity, explicit sex, and controversial politics kept this film from being shown in the US while its seizure by Customs was appealed. The film is a narrative yet it's a documentary that shows us the behind-the- scenes world of the filmmakers during the narrative, the fourth wall being broken. This film is the most direct possible way of making a movie I have ever seen. The movie predominantly works as a time capsule of 1960s psychedelic goings-on, freedom-fighting and sexual liberation. I like to think of it as much more than that.<br /><br />I didn't think I would want to waste my time with the blue version of this movie, but I actually really do. This film is a buried treasure. Give it a try."

sample1 = remove_special(sample, True, True)
sample2 = remove_special(sample,False,True)
sample3 = remove_special(sample,True,False)
sample4 = remove_special(sample, False, False)
print(sample1)
print(sample2)
print(sample3)
print(sample4)

I didnt expect to like this film as much as I did I got it simply because I saw it on the list of Top  Most Controversial Films of All Time It didnt look particularly great I was pleasantly surprised to find that it was one of the most cleverly composed films of recent memorybr br Its about a twentyyearold woman wants to know everything She stores every bit of information she collects in an enormous archive She experiments with experience in sex political activism and human relationships Meanwhile films crew is shown making the film and we view their reactions to the story and each other Nudity explicit sex and controversial politics kept this film from being shown in the US while its seizure by Customs was appealed The film is a narrative yet its a documentary that shows us the behindthe scenes world of the filmmakers during the narrative the fourth wall being broken This film is the most direct possible way of making a movie I have ever seen The movie predominantly works as a time ca

In [39]:
#view stopword list
stop = stopwords.words('english')
print(stop)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [40]:
#define my own stopword list to exclude those stopwards that could change meaning of a sentence, eg. can't, doesn't
stopword_list = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", 
                 "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 
                 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 
                 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 
                 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 
                 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 
                 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 
                 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 
                 'up', 'down', 'in', 'out', 'on', 'off', 'further', 'then', 'here', 'there', 'when', 'where', 
                 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more','other', 'some', 'such', 'own', 
                 'same', 'so', 'than', 'too', 's', 't', 'can', 'will', 'just', 'now', 'd', 'll', 'm', 'o', 
                 're', 've', 'y', ]
print(stopword_list)

def remove_stopword(text):
  result = ' '.join([i for i in text.split() if not i in stopword_list])
  return result

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'further', 'then', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'other', 'some', 'such', 'own', 'same', 'so', 'than', 'too', 's', 't', 'can', 'will', 'just', 'now', 'd', 'll', 'm', 'o', 're', 've', 'y

In [41]:
#Lemmatization
def lemmatize(text):
  lemmatizer = WordNetLemmatizer()
  tokens = word_tokenize(text)
  result = ' '.join([lemmatizer.lemmatize(w) for w in tokens])
  return result

In [42]:
#remove single character as it generally has less meaning
def remove_single_char(text):
  return re.sub(r'\b\w\b', '', text)

In [43]:
#reduce multiple whitespaces to single space
def reduce_space(text):
  return re.sub(r'\s+', ' ', text).strip()

In [44]:
sample = "I didn't expect to like this film as much as I did. I got it simply because I saw it on the list of Top 25 Most Controversial Films of All Time. It didn't look particularly great. I was pleasantly surprised to find that it was one of the most cleverly composed films of recent memory.<br /><br />It's about a twenty-year-old woman wants to know everything. She stores every bit of information she collects in an enormous archive. She experiments with experience in sex, political activism, and human relationships. Meanwhile, film's crew is shown making the film and we view their reactions to the story and each other. Nudity, explicit sex, and controversial politics kept this film from being shown in the US while its seizure by Customs was appealed. The film is a narrative yet it's a documentary that shows us the behind-the- scenes world of the filmmakers during the narrative, the fourth wall being broken. This film is the most direct possible way of making a movie I have ever seen. The movie predominantly works as a time capsule of 1960s psychedelic goings-on, freedom-fighting and sexual liberation. I like to think of it as much more than that.<br /><br />I didn't think I would want to waste my time with the blue version of this movie, but I actually really do. This film is a buried treasure. Give it a try."

sample = remove_single_char(sample)

print(sample)
print(reduce_space(sample))

 didn' expect to like this film as much as  did.  got it simply because  saw it on the list of Top 25 Most Controversial Films of All Time. It didn' look particularly great.  was pleasantly surprised to find that it was one of the most cleverly composed films of recent memory.<br /><br />It' about  twenty-year-old woman wants to know everything. She stores every bit of information she collects in an enormous archive. She experiments with experience in sex, political activism, and human relationships. Meanwhile, film' crew is shown making the film and we view their reactions to the story and each other. Nudity, explicit sex, and controversial politics kept this film from being shown in the US while its seizure by Customs was appealed. The film is  narrative yet it'  documentary that shows us the behind-the- scenes world of the filmmakers during the narrative, the fourth wall being broken. This film is the most direct possible way of making  movie  have ever seen. The movie predominantly

In [45]:
def clean_text(text, remove_punc = False, remove_digits = False):
  text = convert_to_lower(text)
#  print(text)
  text = remove_newline(text)
#  print(text)
  text = remove_link(text)
#  print(text)
  text = decontract(text)
#  print(text)
  text = remove_special(text, remove_punc, remove_digits)
#  print(text)
  text = remove_stopword(text)
#  print(text)  
#  text = lemmatize(text)
#  print(text)
  text = remove_single_char(text)
#  print(text)
  text = reduce_space(text)
#  print(text)

  return text

In [46]:
punc = False
dig = False

In [47]:
exam_raw['Clean_text'] = exam_raw[:][0].apply(clean_text, remove_punc = punc, remove_digits = dig)

In [48]:
exam_raw.head()

,0,Clean_text
0,review 1 x y z,review
1,review 2 x y z,review
2,review 3 x y z,review
3,review 4 x y z,review
4,review 5 x y z,review


##load model

In [11]:
filename_vec = '/content/NLP-Assn-2/vectorizer.pickle'
pickle.load(open(filename_vec, 'rb'))

Cloning into 'NLP-Assn-2'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 11 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (11/11), done.


In [49]:
filename = '/content/NLP-Assn-2/model_1.sav'
model = pickle.load(open(filename, 'rb'))

##predict

In [53]:
vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)
exam_vectors = vectorizer.transform(exam_raw['Clean_text'])

NotFittedError: ignored

In [ ]:
pred = model.predict(exam_vectors)


In [ ]:
print(pred)

In [ ]:
#output to df
exam['Pred_label'] = pred

In [ ]:
f1_score = sk.metrics.f1_score(list(test['Label']), prediction_linear)
print(round(f1_score, 2))
print(f1_score)